In [ ]:
from __future__ import print_function
import os.path
import pandas as pd
import gzip
import sys
import numpy as np

sys.path.insert(0, '..')

from src.CCLE_postp_function import *
from JKBio import Datanalytics as da 
from JKBio import TerraFunction as terra
from JKBio import Helper as h
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm

from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier

from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict
from IPython.display import Image,display



%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

In [ ]:
samplesetname = "20Q2"
prevname="20Q1"
prevprevname ='19Q4'
prevprevversion= 18
virtual_public='public-20q2-075d'
virtual_dmc='dmc-20q2-2db6'
virtual_internal='internal-20q2-7f46'

workspace1="broad-genomics-delivery/Getz_IBM_CellLines_Exomes"
workspace2="broad-firecloud-ccle/CCLE_DepMap_WES"
workspace3="broad-genomics-delivery/CCLE_DepMap_WES"

workspace6="terra-broad-cancer-prod/CCLE_DepMap_WES"

refworkspace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline"

rnaworkspace="broad-firecloud-ccle/DepMap_hg38_RNAseq"

source1="ibm"
source2="ccle"
source3="ccle"
source6="ccle"
source7="ibm"

refsheet_url = "https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"

release = samplesetname

In [ ]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)
wm3 = dm.WorkspaceManager(workspace3)

wm6 = dm.WorkspaceManager(workspace6)

refwm = dm.WorkspaceManager(refworkspace)

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame()

In [ ]:
! wget https://github.com/mskcc/vcf2maf/archive/v1.6.18.zip

In [ ]:
# for rnaseq
#! gsutil cp gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/8af15313-7faf-4344-a7b7-97dd52353dc8/RNAseq/0de1da92-8821-4cbd-a9ce-96ed8b06f395/call-MergeVCFs/G41677.MV-4-11.5.g.vcf.gz temp/
path= "temp/G41677.MV-4-11.5.g.vcf.gz"
rnaseq, description = h.vcf_to_df(path)

In [ ]:
# for rnaseq filtered
#!gsutil cp gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/8af15313-7faf-4344-a7b7-97dd52353dc8/RNAseq/0de1da92-8821-4cbd-a9ce-96ed8b06f395/call-VariantFiltration/G41677.MV-4-11.5.variant_filtered.vcf.gz temp/
rnaseqfilt, description = h.vcf_to_df("temp/G41677.MV-4-11.5.variant_filtered.vcf.gz")

In [ ]:
# for unfiltered wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/51fcb10c-9a67-4ff9-b13c-d9667e2d49d0/HaplotypeCallerGvcf_GATK4/6611b28a-12c2-484e-a4a3-901708785f95/call-MergeGVCFs/SANGER_MV411_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE.g.vcf.gz temp/
wes, description = h.vcf_to_df("temp/SANGER_MV411_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE.g.vcf.gz")

In [ ]:
# for filtered wes
#! gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/d953cc3f-7941-4426-bb44-a41992d57606/Cram2FilteredVcf/dc943e9d-aa73-4314-b43d-208a80319997/call-FilterVariantTranches/CDS-FInR9b_cnn_filtered.vcf.gz temp/
wesfilt, description = h.vcf_to_df("temp/CDS-FInR9b_cnn_filtered.vcf.gz")

In [ ]:
# for somatic filtered wes
somwesfilt = pd.read_csv('temp/mutations.'+release+'.all.tsv', sep='\t')
somwesfilt = somwesfilt[somwesfilt.DepMap_ID=="ACH-000045"]
somwesfilt = somwesfilt.rename(columns=h.rename_mut)

In [ ]:
# for somatic unfiltered wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Oncotate_Task/CDS-FInR9b.MuTect1_SNV.MuTect2_INDEL.Strelka_INDEL.annotated.maf temp/
somwes = pd.read_csv("temp/CDS-FInR9b.MuTect1_SNV.MuTect2_INDEL.Strelka_INDEL.annotated.maf", sep='\t')
somwes = somwes.rename(columns=h.rename_mut)

In [ ]:
# for strelka wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Strelka/CDS-FInR9b.all.somatic.snvs.vcf temp/
strelka, description = h.vcf_to_df("temp/CDS-FInR9b.all.somatic.snvs.vcf", hasfilter=True,samples=['sample1_A','sample1_B'])
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Strelka/CDS-FInR9b.all.somatic.indels.vcf temp/
strelka2, description = h.vcf_to_df("temp/CDS-FInR9b.all.somatic.indels.vcf",samples=['sample1_A','sample1_B'],hasfilter=True)
strelka = pd.concat([strelka,strelka2], ignore_index=True,sort=False).sort_values(by=['chr','pos'])

In [ ]:
# for mutect1 wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Gather_Task/CDS-FInR9b.MuTect1.call_stats.txt temp/
mutect1 = pd.read_csv('temp/CDS-FInR9b.MuTect1.call_stats.txt', sep='\t', header=1, index_col=None)
mutect1 = mutect1.rename(h.rename_mut)

In [ ]:
# for mutect 2 wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Gather_Task/CDS-FInR9b.MuTect2.call_stats.vcf temp/
mutect2, description = h.vcf_to_df('temp/CDS-FInR9b.MuTect2.call_stats.vcf', hasfilter=True)

In [ ]:
# for unfiltered wgs
# for filtered wgs
#! gsutil cp gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/77dbe69a-509c-4618-a2bf-f809c1aea47a/liftover_workflows/2950a03d-c092-4355-b261-2e6eb50fca41/call-liftover/ccle_ACH-000045_liftover.vcf temp/
wgsfilt, description = h.vcf_to_df('temp/ccle_ACH-000045_liftover.vcf')

In [ ]:
data= {'wgsfilt': wgsfilt, 'mutect2': mutect2, 'mutect1': mutect1, 'strelka': strelka, 'somwes': somwes, 'somwesfilt': somwesfilt, 'wes': wes, 'rnaseq': rnaseq, 'rnaseqfilt': rnaseqfilt, 'wesfilt': wesfilt}

In [ ]:
mutset = {}
for i,v in data.items():
    chrom = v.chr.tolist()
    alt = v.alt.tolist()
    pos = v.pos.tolist()
    mutset[i] = set([str(chrom[i])+'_'+str(pos[i]) for i, a in enumerate(alt)])

In [ ]:
len(mutset['wgsfilt'] & mutset['wesfilt']) / len(mutset['wesfilt']), len(mutset['wgsfilt']), len(mutset['wesfilt'])

In [ ]:
len(mutset['rnaseqfilt']  & mutset['wesfilt']) / len(mutset['wesfilt'])

In [ ]:
len(mutset['rnaseq']  & mutset['wesfilt']) / len(mutset['wesfilt'])

In [ ]:
len(mutset['rnaseqfilt']  & mutset['rnaseq']) / len(mutset['rnaseq'])

In [ ]:
len(mutset['wes']  & mutset['wesfilt']) / len(mutset['wesfilt'])

In [ ]:
len(mutset['wes']  & mutset['wesfilt']) / len(mutset['wes'])
# how many mutations is the cnn filtering..

In [ ]:
len(mutset['wesfilt']), len(mutset['wes'])

In [ ]:
len(mutset['strelka']), len(mutset['mutect1']), len(mutset['mutect2'])
# how many mutations each contain and their similarity between each caller in cga

In [ ]:
len(mutset['strelka'] & mutset['mutect1'] & mutset['mutect2'])

In [ ]:
len(mutset['somwes']), len(mutset['somwesfilt']), len(mutset['somwesfilt'] & mutset['somwes'])
# how many mutations each contain and their similarity between cga middle and cga end

In [ ]:
len(mutset['somwes'] & mutset['mutect1']) / len(mutset['mutect1']), len(mutset['somwesfilt'] & mutset['mutect1']) / len(mutset['mutect1'])

In [ ]:
len(mutset['somwes'] & mutset['mutect2']) / len(mutset['mutect2']), len(mutset['somwesfilt'] & mutset['mutect2']) / len(mutset['mutect2'])

In [ ]:
len(mutset['somwes'] & mutset['strelka']) / len(mutset['strelka']), len(mutset['somwesfilt'] & mutset['strelka']) / len(mutset['strelka'])

In [ ]:
len(mutset['somwes'] & mutset['wes']) / len(mutset['somwes']), len(mutset['wes'])

In [ ]:
len(mutset['somwes'] & mutset['wesfilt']) / len(mutset['somwes']), len(mutset['wesfilt'])

In [ ]:
len(mutset['somwesfilt'] & mutset['wesfilt']) / len(mutset['somwesfilt']), len(mutset['wesfilt'])
# good agreement between wes hapcall and wes cga and we can see how much we recover once bad mutations get filtered!

In [ ]:
len(mutset['somwes'] & mutset['wesfilt']), len(mutset['somwesfilt'] & mutset['wesfilt'])

In [ ]:
len(mutset['somwes'] & mutset['wgsfilt']) / len(mutset['somwes'])

In [ ]:
len(mutset['somwesfilt'] & mutset['wgsfilt']) / len(mutset['somwesfilt'])
# good agreement between wgs hapcall and wes cga

In [ ]:
len(mutset['somwesfilt'] & mutset['wes']) / len(mutset['somwesfilt']), len(mutset['wes'])
# good agreement between wes hapcall and wes cga

In [ ]:
#we don't have a good similarity wes/wgs, can it be because of slight differences in the base distances? can it be because of the mutation caller?
len(set([i for i in mutset['wesfilt']]) & set([i for i in mutset['wgsfilt']]))/len(mutset['wesfilt'])

In [ ]:
def locify(dat):
    return ['chr'+d.split('_')[0]+':'+str(int(d.split('_')[1])-40)+'-'+str(int(d.split('_')[1])+40) for d in dat]

In [ ]:
'chr6:27420132-27420212':'rna',
'chr5:32089849-32089929':'del',
'chr4:100574667-100574747':'wgs',
'chr7:100643254-100643334':'wgs',
'chr15:89070794-89070874':'wgs',
'chr19:36120039-36120119':'snp',
'chr20:31196288-31196368':'snp',
'chr15:42343066-42343146':False,
'chr11:6568654-6568734':'snp',

'chr9:24543236-24543316':'snp',
'chr15:23261902-23261982':'snp',
'chr4:84255808-84255888':True,
'chr13:76055619-76055699':'snp',
'chrX:18969242-18969322':,
'chr4:36152613-36152693':,
'chr19:50354695-50354775':,
'chr4:100574668-100574748':,
'chr6:110763839-110763919':,
'chr7:100648111-100648191':,
'chr5:70858221-70858301':,
'chr3:120489593-120489673':,
'chr2:27601639-27601719':,
'chr4:77184865-77184945':,

In [ ]:
locify(mutset['somwesfilt'] - mutset['wesfilt'])

In [ ]:
def subset(df,mut):
    return pd.concat([df[(df.chr==val.split('_')[0]) & (df.pos==int(val.split('_')[1]))] for val in mut])

In [ ]:
a = subset(data['somwesfilt'],mutset['somwesfilt'] - mutset['wesfilt'])
print(60-a.SangerRecalibWES_AC.isna().sum())
a[~a.SangerRecalibWES_AC.isna()]

####  we have 60-40=20 that the old pipeline has found but not us. of these 20; 7 were suspicious as they were not confirmed by any other sequencing methods, had low depth etc.

In [ ]:
res = subset(data['somwesfilt'],mutset['somwesfilt'] & mutset['wesfilt'])
print(res.SangerRecalibWES_AC.isna().sum())
res[res.SangerRecalibWES_AC.isna()]

In [ ]:
# we have found 16 mutations that the old pipeline did not fouund, of these, only one was only confirmed by one sequencing method

In [ ]:
subset(data['somwesfilt'],mutset['rnaseqfilt'] & mutset['somwesfilt']).RNAseq_AC.isna().sum() #113

In [ ]:
subset(data['somwesfilt'],mutset['somwesfilt'] - mutset['rnaseqfilt']).RNAseq_AC.isna().sum() #113

In [ ]:
# we have found 12 more rnaseq mutations and lost 9 rnaseq mutations

In [ ]:
res = subset(data['somwesfilt'],mutset['somwesfilt'] - mutset['wgsfilt'])
print(37-res.WGS_AC.isna().sum())

In [ ]:
res

### we have high allele freq mut which seems it might be that they get filtered by the cnn. two seemed quite shady